## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,59.88,89,75,5.99,light rain
1,1,Brae,GB,60.3964,-1.3530,46.31,91,52,2.42,broken clouds
2,2,Sangar,RU,63.9242,127.4739,70.47,29,53,4.36,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,67.05,74,83,11.88,broken clouds
4,4,Jodar,ES,37.8406,-3.3526,68.99,61,78,3.89,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Rikitea,PF,-23.1203,-134.9692,76.77,77,76,20.74,broken clouds
7,7,Butaritari,KI,3.0707,172.7902,82.89,79,100,17.05,light rain
16,16,Dwarka,IN,22.2394,68.9678,83.43,78,100,14.14,overcast clouds
22,22,Saint-Louis,SN,16.3333,-15.0000,77.25,40,95,14.20,overcast clouds
24,24,Kefamenanu,ID,-9.4467,124.4781,78.55,69,86,8.99,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
preferred_cities_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
count,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000
mean,369.343348,9.879662,22.232259,81.344292,64.824034,55.317597,8.309485
std,201.756880,16.306154,94.475965,3.818948,22.368368,36.988844,5.364424
min,6.000000,-23.778100,-175.200000,75.070000,7.000000,0.000000,0.000000
25%,195.000000,-2.574400,-58.616700,78.060000,55.000000,20.000000,4.340000
50%,382.000000,10.566900,37.592800,81.280000,72.000000,62.000000,6.910000
75%,540.000000,21.444700,109.483300,84.070000,79.000000,94.000000,11.860000
max,718.000000,42.800000,172.790200,89.780000,100.000000,100.000000,23.020000


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,76.77,broken clouds,-23.1203,-134.9692,
7,Butaritari,KI,82.89,light rain,3.0707,172.7902,
16,Dwarka,IN,83.43,overcast clouds,22.2394,68.9678,
22,Saint-Louis,SN,77.25,overcast clouds,16.3333,-15.0000,
24,Kefamenanu,ID,78.55,overcast clouds,-9.4467,124.4781,
38,Marsh Harbour,BS,76.77,clear sky,26.5412,-77.0636,
48,Cam Ranh,VN,89.56,few clouds,11.9214,109.1591,
49,Hithadhoo,MV,82.71,overcast clouds,-0.6000,73.0833,
56,Barra Dos Coqueiros,BR,78.82,few clouds,-10.9089,-37.0386,
57,Kapaa,US,81.12,few clouds,22.0752,-159.3190,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"   
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]  
        if hotel_df.loc[index, "Hotel Name"] == '':
            print(hotels["results"])
    except(IndexError,KeyError):
        print("Hotel not found ... skipping")
        pass    
        

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [10]:
clean_hotel_df=hotel_df.replace("",np.nan)
clean_hotel_df.count()

City                   233
Country                233
Max Temp               233
Current Description    233
Lat                    233
Lng                    233
Hotel Name             211
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_drop_df=clean_hotel_df.dropna()
clean_hotel_drop_df.count()


City                   211
Country                211
Max Temp               211
Current Description    211
Lat                    211
Lng                    211
Hotel Name             211
dtype: int64

In [12]:
clean_hotel_drop_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,76.77,broken clouds,-23.1203,-134.9692,Pension Maro'i
7,Butaritari,KI,82.89,light rain,3.0707,172.7902,Isles Sunset Lodge
16,Dwarka,IN,83.43,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
24,Kefamenanu,ID,78.55,overcast clouds,-9.4467,124.4781,Sokowindo Losmen
38,Marsh Harbour,BS,76.77,clear sky,26.5412,-77.0636,Abaco Beach Resort


In [13]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_drop_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# Note: Description in Canvas and here say max temp instead of hote name. The graphic shows hotel name and not max temp.
# so I put in both
info_box_template = """
<dl>
<dt> Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_drop_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_drop_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))